<a href="https://colab.research.google.com/github/shukiBenishty/mars/blob/main/non_linreg_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import io
from datetime import *

# Prepare logs directory for Tensorboard

In [3]:
# Clear out any prior log data.
!rm -rf logs

logdir = "logs/linreg_keras/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Load data

In [4]:
from google.colab import files
import os.path

if(not os.path.isfile("ds.csv") ):
  uploader = files.upload()
  io.StringIO(uploader['ds.csv'].decode('utf-8'))

ds = np.genfromtxt('ds.csv', delimiter=',')

In [5]:
ds_len = ds.shape[0]

shuffled = tf.random.shuffle(ds)
train_len =  int(ds_len * 0.7)
test_len = int(ds_len * 0.3)
validation_len = ds_len - (train_len + test_len)


ds_train, ds_test, ds_validation = tf.split(shuffled, [train_len, test_len, validation_len])

X_train, Y_train = np.hsplit(ds_train, 2)
X_test, Y_test = np.hsplit(ds_test, 2)
X_validation, Y_validation = np.hsplit(ds_validation, 2)


# x_len = 1000

# X = tf.random.uniform(shape=(x_len, 1), minval=-10 , maxval=10)

# _X = tf.random.shuffle(X)
# X_train, X_test, X_validation = tf.split(_X, [int(x_len * 0.7), int(x_len *0.2), int(x_len *0.1)])

# def target(x):
#   noise =  tf.random.normal(shape=(len(x),1) , mean=0.0, stddev=0.2)

#   # return (x**4) + (-0.5 * x) +5 + noise
#   return np.sin(x*2) * x + 5

# Y_train = target(X_train)
# Y_test = target(X_test)
# Y_validation = target(X_validation)

In [6]:
def gen_plot(figure):
    """Create a pyplot plot and save to buffer."""
    buf = io.BytesIO()
    figure.savefig(buf, format='png')
    buf.seek(0)
    return buf


# Create and train the model

In [ ]:
#base on cybenko theorem

tensorboardCallback = tf.keras.callbacks.TensorBoard() 
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping( 
    min_delta=1e-5,
    mode="min",
    restore_best_weights=True,
    patience=30, verbose=1
)

model = keras.Sequential([
  keras.layers.Dense(128,  activation=keras.activations.sigmoid),
  keras.layers.Dense(1)
])
model.compile(loss=tf.losses.mean_squared_error,
              optimizer=tf.optimizers.Adam(0.01))
model.fit(X_train, Y_train, epochs=10000, 
          batch_size=100,
          verbose=True,
  callbacks=[
    tensorboardCallback,
    # earlyStoppingCallback
])

model.summary()
print(model.metrics_names)

Epoch 1/10000
2/2 [==============================] - 1s 408ms/step - loss: 4824.9170
Epoch 2/10000
2/2 [==============================] - 0s 6ms/step - loss: 4551.9644
Epoch 3/10000
2/2 [==============================] - 0s 5ms/step - loss: 4459.4722
Epoch 4/10000
2/2 [==============================] - 0s 6ms/step - loss: 4209.0560
Epoch 5/10000
2/2 [==============================] - 0s 4ms/step - loss: 4011.5673
Epoch 6/10000
2/2 [==============================] - 0s 3ms/step - loss: 3828.2161
Epoch 7/10000
2/2 [==============================] - 0s 4ms/step - loss: 3542.2939
Epoch 8/10000
2/2 [==============================] - 0s 5ms/step - loss: 3223.8627
Epoch 9/10000
2/2 [==============================] - 0s 5ms/step - loss: 3277.7574
Epoch 10/10000
2/2 [==============================] - 0s 7ms/step - loss: 3024.3373
Epoch 11/10000
2/2 [==============================] - 0s 4ms/step - loss: 2877.0087
Epoch 12/10000
2/2 [==============================] - 0s 5ms/step - loss: 2792.5561

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.title = "Results"
ax[0].scatter(X_train, Y_train)

ax[1].scatter(X_train, Y_train)
ax[1].scatter(X_train, model.predict(X_train), color='red')

# Prepare the plot
plot_buf = gen_plot(fig)
# Convert PNG buffer to TF image
image = tf.image.decode_png(plot_buf.getvalue(), channels=4) 
image = tf.expand_dims(image, 0)

#write to log
formatted_vals = [ "var: {}, value: {}".format(v.name, v.numpy()) for v in model.trainable_variables]
with file_writer.as_default():
  tf.summary.image("Results", image, step=0)
  tf.summary.text("Trainable variables names: ", 
                formatted_vals,
                step=0)



In [ ]:
%tensorboard --logdir logs